In [6]:
from manim import *
import numpy as np

quality = "ql"

Multiple scene-side constants:

In [7]:
# Font sizes
SMALL_FONT = 24
MED_FONT = 32
LARGE_FONT = 48

# Colors
C_HIGHLIGHT = TEAL_C
C_EMPHASIS = ORANGE

Create a PID simulation that we can use for future scenes. The scenario accounts
for friction and a constant force, and effort means acceleration. Disclaimers to
this model should be made in animations: motors output is closer to torque, not
acceleration, and doesn't even do that in reality.

In [8]:
class PIDSim():
    """
    PID controller simulator. Assumes a system with a frictive accleration 
    that swaps on sign(vel) and a constant (i.e. graviational) acceleration,
    and where effort is acceleration. The target is always 0: use a coordinate
    system that accounts for this.
    """

    def __init__(self, kp, ki, kd,
                dt = 0.01, # default, may be overridden by caller 
                ak_friction = 0, a_const = 0, 
                start_pos = -1,  start_vel = 0):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        
        self.ak_friction = ak_friction
        self.a_const = a_const
        
        self.dt = dt
        
        self.reset(start_pos, start_vel)

    def reset(self, pos, vel):
        self.pos = pos
        self.vel = vel
        self.integral = 0
        self.last_error = None

    def step(self, dt = None):
        """
        Simulates the system with one sample over given time dt. If dt is None,
        uses the default dt.

        Returns total accleration (effort) along with each of its components:
        a_p, a_i, a_d, a_friction, a_const
        """
        if dt is None:
            dt = self.dt
        
        error = 0 - self.pos
        self.integral += error * dt
        if self.last_error is None:
            derivative = 0
        else:
            derivative = (error - self.last_error) / dt
        self.last_error = error

        a_p = self.kp * error
        a_i = self.ki * self.integral
        a_d = self.kd * derivative
        effort = a_p + a_i + a_d
        a_friction = -self.a_friction * np.sign(self.vel)
        dv_friction = a_friction * dt
        a_other = self.a_const + effort
        dv_other = a_other * dt

        v_other = self.vel + dv_other
        # hack to prevent friction-induced: bound friction to not switch sign of 
        # v_other
        if(np.sign(v_other + dv_friction) != np.sign(self.vel)):
            dv_friction = 0
        
        self.vel = v_other + dv_friction
        self.pos += self.vel * dt
        return effort, a_p, a_i, a_d, a_friction, self.a_const

Class for vanilla PID animation scenes. Meant to provide convenient ways of 
handling specific scene objects, such as showing and hiding a bracket demonstrating
how we measure error.

In [9]:
%%manim -v WARNING --disable_caching -$quality PIDScene
        
class PIDScene(Scene):

    LINE_Y = -2
    LINE_WIDTH = 11

    ROBOT_SIZE = 1.5
    ROBOT_Y = LINE_Y + ROBOT_SIZE / 2 + 0.25

    def construct(s):
        self = s

        # ---------------------------------------------------------------------
        # simulation number line and labels
        #
        s.sim_line = NumberLine(
            x_range = [-1, 1, 1],
            length = s.LINE_WIDTH,
            include_numbers = False
        )
        s.sim_line.move_to([0, s.LINE_Y, 0])

        s.line_labels = ["target - 1", "target", "target + 1"]
        s.line_labels = [Text(label, font_size = SMALL_FONT) for label in s.line_labels]
        s.line_labels[1].set_color(C_EMPHASIS)
        s.sim_line.add_labels({i-1 : s.line_labels[i] for i in range(3)},
                             font_size = SMALL_FONT)
       
        # ---------------------------------------------------------------------
        # Robot model: a square and a dot
        #
        s.robot_pos = ValueTracker(-1) # position on number line
        s.robot = Square(side_length = s.ROBOT_SIZE)
        s.robot.add_updater(lambda x: x.next_to(s.sim_line.n2p(s.robot_pos.get_value()), UP))
        s.robot_dot = Dot(color = C_HIGHLIGHT)
        s.robot_dot.add_updater(lambda x: x.move_to(s.robot.get_center()))
        
        s.robot_label = Text("(Robot)", color = WHITE, font_size= MED_FONT)
        s.robot_label.add_updater(lambda x: x.next_to(s.robot, UP))
        
        # ---------------------------------------------------------------------
        # Error bracket
        #
        def draw_error_bracket():
            to_point = s.robot.get_center()
            to_point[1] += s.ROBOT_SIZE / 2
            error_bracket = BraceBetweenPoints(to_point, s.sim_line.n2p(0), direction=UP)
            return error_bracket
        s.error_bracket = always_redraw(draw_error_bracket)
        

        # Test animations
        s.play(s.show_line())
        # s.play(s.annotate_line())
        s.play(s.show_robot())
        s.play(s.annotate_robot())
        s.play(s.unannotate_robot())
        s.play(Create(s.error_bracket))
        s.play(s.robot_pos.animate.set_value(0.5), run_time = 3)
        s.wait(3)

    def show_line(self, t = 1):
        return Create(self.sim_line, run_time = t)

    def hide_line(self, t = 1):
        return FadeOut(self.sim_line, run_time = t)

    def show_robot(self, t = 1):
        return AnimationGroup(
            Create(self.robot, run_time = t),
            Create(self.robot_dot, run_time = t),
            lag_ratio = 0.2,
            run_time = t
        )

    def hide_robot(self, t = 1):
        return AnimationGroup(
            FadeOut(self.robot, run_time = t),
            FadeOut(self.robot_dot, run_time = t),
            lag_ratio = 0.2,
            run_time = t
        )
    
    def annotate_robot(self, t = 1):
        return Write(self.robot_label, run_time = t)

    def unannotate_robot(self, t = 1):
        return FadeOut(self.robot_label, run_time = t)

    def move_robot(self, x, t = 1):
        return self.robot_pos.animate.set_value(x, run_time = t)

    def draw_error_bracket(self):
        return Create(self.error_bracket)
    
    def hide_error_bracket(self):
        return FadeOut(self.error_bracket)

FileNotFoundError: [Errno 2] No such file or directory: 'media/jupyter/PIDScene@2023-03-03@05-31-57.mp4'